**Общая информация**

**Срок сдачи:** 16 марта 2020, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии фа
йла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1318]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml, fetch_20newsgroups

from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier, KDTree
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

from scipy.spatial import distance
from scipy.stats import mode

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [1319]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
    def fit(self, X, y):
        if self.algorithm == 'brute':
            self.X = X
            self.y = y
        elif self.algorithm == 'kd_tree':
            self.tree = KDTree(X, metric = 'minkowski', p=2)
            self.y = y
        else:
            pass
    def predict(self, X):
        if self.algorithm == 'brute':
            ro = cdist(X, self.X, metric = 'minkowski', p=2)
            ind = np.argsort(ro, axis=1)
            brute_res = self.y[ind[:,:self.n_neighbors]]
            brute_res = mode(brute_res,1)[0].flatten()
            return brute_res
        elif self.algorithm == 'kd_tree':
            kn = self.tree.query(X, self.n_neighbors, return_distance = False)
            kdt_res = self.y[kn]
            kdt_res = mode(kdt_res,1)[0].flatten()
            return kdt_res
        else:
            pass
        return None
    def score(self,X,y):
        return accuracy_score(y, self.predict(X))

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [1320]:
iris = datasets.load_iris()

In [1321]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [1322]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [1323]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [1324]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
my_clf_pred
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [1325]:
%time clf.fit(X_train, y_train)

Wall time: 0 ns


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [1326]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [1327]:
%time clf.predict(X_test)

Wall time: 2.99 ms


array([2, 0, 2, 0, 1, 1, 2, 0, 0, 1, 2, 2, 0, 1, 1])

In [1328]:
%time my_clf.predict(X_test)

Wall time: 1.52 ms


array([2, 0, 2, 0, 1, 1, 2, 0, 0, 1, 2, 2, 0, 1, 1])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [1329]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [1330]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [1331]:
%time clf.fit(X_train, y_train)

Wall time: 998 µs


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [1332]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [1333]:
%time clf.predict(X_test)

Wall time: 2 ms


array([2, 1, 0, 0, 1, 1, 1, 0, 2, 1, 1, 0, 2, 1, 0])

In [1334]:
%time my_clf.predict(X_test)

Wall time: 1.99 ms


array([2, 1, 0, 0, 1, 1, 1, 0, 2, 1, 1, 0, 2, 1, 0])

In [1335]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [1336]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [1337]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [1338]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)
y = target

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [1339]:
kf = KFold(n_splits=3, shuffle = True)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [    0     1     2 ... 11310 11312 11313] TEST: [    7    10    11 ... 11308 11309 11311]
TRAIN: [    1     2     7 ... 11310 11311 11313] TEST: [    0     3     4 ... 11300 11305 11312]
TRAIN: [    0     3     4 ... 11309 11311 11312] TEST: [    1     2     8 ... 11306 11310 11313]


In [1342]:
neighbors = list(range(1,10))
for K in neighbors:
    knn = KNeighborsClassifier(n_neighbors = K)
    print(cross_val_score(knn, X, target, cv = 3).mean())

0.2059397033733749
0.18295912162539008
0.18128001279321015
0.18225269702110738
0.1830479841929615
0.18349016451667058
0.18614169980031234
0.18684878056279441
0.1907378302104421


Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [1023]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.